In [7]:
import wandb
wandb.login() 
# 8599fbb702cb5767e13d2ac3b1cdcc1c9b65d451

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc


True

In [11]:
from tem_dataloader import MultimodalDataset
trained_model_name = "unimodel_0"
environment_name = 'AbandonedCableExposure'

my_dataset = MultimodalDataset(f'/mnt/data/tartanairv2filtered/{environment_name}/Data_easy')

Processing folder: /mnt/data/tartanairv2filtered/AbandonedCableExposure/Data_easy/P007
Processing folder: /mnt/data/tartanairv2filtered/AbandonedCableExposure/Data_easy/P003
Processing folder: /mnt/data/tartanairv2filtered/AbandonedCableExposure/Data_easy/P005
Processing folder: /mnt/data/tartanairv2filtered/AbandonedCableExposure/Data_easy/P004
Processing folder: /mnt/data/tartanairv2filtered/AbandonedCableExposure/Data_easy/P000
Processing folder: /mnt/data/tartanairv2filtered/AbandonedCableExposure/Data_easy/P002
Processing folder: /mnt/data/tartanairv2filtered/AbandonedCableExposure/Data_easy/P006
Processing folder: /mnt/data/tartanairv2filtered/AbandonedCableExposure/Data_easy/P001
Number of images: 25448
Number of depth: 25448
Number of lidar: 25448


In [12]:
from torch.utils.data import DataLoader
batch_size = 32
train_dataloader = DataLoader(my_dataset, batch_size=batch_size, num_workers=4, shuffle=True)

In [13]:
from transformers import ViTMAEForPreTraining, ViTMAEConfig
from transformers.models.vit_mae.modeling_vit_mae import ViTMAEModel
import torch

# create Unimodel ViT
model_name = "facebook/vit-mae-base"
vit_config = ViTMAEConfig.from_pretrained(model_name)
vit_config.output_hidden_states=True
vit_model = ViTMAEModel.from_pretrained(model_name,config=vit_config)

config = ViTMAEConfig.from_pretrained(model_name)
config.output_hidden_states=True

# load from pretrained model and replace the original encoder with custom encoder
unimodel = ViTMAEForPreTraining.from_pretrained(model_name,config=config)
unimodel.vit = vit_model

output_dir='/home/ubuntu/weights/' + trained_model_name

# Load the state_dict from the saved model
# state_dict = torch.load(f"{output_dir}/pytorch_model.bin")

# Apply the state_dict to the custom_model
# unimodel.load_state_dict(state_dict)

from transformers.optimization import AdamW
from transformers import get_linear_schedule_with_warmup

epochs = 1
learning_rate = 0.01
weight_decay = 0.01
warmup_steps = 0
total_steps = len(train_dataloader) * epochs
optimizer = AdamW(unimodel.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
unimodel.to(device)


Some weights of the model checkpoint at facebook/vit-mae-base were not used when initializing ViTMAEModel: ['decoder.decoder_layers.6.output.dense.weight', 'decoder.decoder_layers.3.output.dense.weight', 'decoder.decoder_layers.6.attention.attention.value.weight', 'decoder.decoder_layers.4.intermediate.dense.weight', 'decoder.decoder_layers.1.layernorm_before.weight', 'decoder.decoder_layers.7.output.dense.weight', 'decoder.decoder_layers.0.layernorm_before.weight', 'decoder.decoder_layers.5.layernorm_after.bias', 'decoder.decoder_layers.0.attention.attention.query.bias', 'decoder.decoder_layers.6.layernorm_before.weight', 'decoder.decoder_layers.3.attention.attention.key.weight', 'decoder.decoder_layers.4.attention.attention.key.bias', 'decoder.decoder_layers.4.attention.attention.query.bias', 'decoder.decoder_layers.5.layernorm_after.weight', 'decoder.decoder_layers.3.attention.output.dense.bias', 'decoder.decoder_layers.4.output.dense.bias', 'decoder.decoder_layers.0.attention.atten

cuda


ViTMAEForPreTraining(
  (vit): ViTMAEModel(
    (embeddings): ViTMAEEmbeddings(
      (patch_embeddings): ViTMAEPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
    )
    (encoder): ViTMAEEncoder(
      (layer): ModuleList(
        (0): ViTMAELayer(
          (attention): ViTMAEAttention(
            (attention): ViTMAESelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTMAEIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)

In [6]:
import time
def train(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    total_samples = 0
    
    for batch in dataloader:
        if trained_model_name == "unimodel_0":
            pixel_values = batch["pixel_values"].to(device)
        elif trained_model_name == "unimodel_1":
            pixel_values = batch["pixel_values1"].to(device)
        elif trained_model_name == "unimodel_2":
            pixel_values = batch["pixel_values2"].to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values, noise=None)

        loss = outputs.loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item() * pixel_values.size(0)
        total_samples += pixel_values.size(0)

        if total_samples % 32 == 0:
            wandb.log({'loss': loss.item()} )
            print(loss.item())

    return total_loss / total_samples

wandb.init(project="11777",name=trained_model_name+"_"+environment_name+time.strftime("%Y%m%d-%H%M%S"))
for epoch in range(epochs):
    train_loss = train(unimodel, train_dataloader, optimizer, scheduler, device)
    print(f"Epoch: {epoch + 1}, Loss: {train_loss:.4f}")
    unimodel.save_pretrained(output_dir)
wandb.finish()

NameError: name 'train_dataloader' is not defined

In [ ]:
unimodel.save_pretrained(output_dir)